In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, MonthLocator
from scipy.signal import savgol_filter
from statsmodels.tsa.seasonal import STL

Process:
1. Load the Data
2. Filter columns from insitu to be saved
3. Matching AND MERGE date insitu and sentinel
4. add global data: season, biom
5. Decompose
6. Merged decompose and original to one csv file
7. Check Stationary

Define Path

In [7]:
directory_insitu = r"input\NETFLUX"
directory_plot = r"plot"
directory_input = r"input"
directory_sentinel = r"input\Sentinel"
# load list of sites
list_site_file = r"D:\Proyectos2024\Agame\Repository\vlabs\sen2gpp\Input\list_sites_without_Lonzee.xlsx"
df = pd.read_excel(list_site_file)
# df
df_training = df[df['role']=='training']
# df_training

Check Insitu Columns
- only include _F suffixed data, if _F not available then choose MDS

In [8]:
list_col = []
nan_cols = []
files = os.listdir(directory_insitu)
for id in df_training['station id']:
    site = df_training.loc[df_training['station id'] == id, 'site name'].iloc[0]
    # print(site)
    data = pd.read_csv(os.path.join(directory_insitu,f"ICOSETC_{id}_FLUXNET_DD_L2.csv"))
    if 'date.1' in data.columns:
        data.drop(columns = ['date.1'], inplace = True)
    # Replace -999 with NaN
    data.replace(-9999, np.nan, inplace=True)
    # Check which columns contain any NaN values
    nan_columns = data.columns[data.isnull().any()].tolist()
    nan_cols.append(nan_columns)
    # print("Columns with NaN values:", nan_columns)
    data.drop(columns = nan_columns, inplace = True)
    list_col.append(set(data.columns))
    # print(data.columns)
# Initialize the intersection with the first set in the list
intersection = list_col[0]
# Use a for loop to intersect with each subsequent set
for s in list_col[1:]:
    intersection = intersection.intersection(s)
intersection_list = list(intersection)
intersection_list.sort()
print(intersection_list)

['DAY_D', 'DAY_RANDUNC_N', 'GPP_DT_CUT_05', 'GPP_DT_CUT_16', 'GPP_DT_CUT_25', 'GPP_DT_CUT_50', 'GPP_DT_CUT_75', 'GPP_DT_CUT_84', 'GPP_DT_CUT_95', 'GPP_DT_CUT_MEAN', 'GPP_DT_CUT_REF', 'GPP_DT_CUT_SE', 'GPP_DT_CUT_USTAR50', 'GPP_DT_VUT_05', 'GPP_DT_VUT_16', 'GPP_DT_VUT_25', 'GPP_DT_VUT_50', 'GPP_DT_VUT_75', 'GPP_DT_VUT_84', 'GPP_DT_VUT_95', 'GPP_DT_VUT_MEAN', 'GPP_DT_VUT_REF', 'GPP_DT_VUT_SE', 'GPP_DT_VUT_USTAR50', 'GPP_NT_CUT_05', 'GPP_NT_CUT_16', 'GPP_NT_CUT_25', 'GPP_NT_CUT_50', 'GPP_NT_CUT_75', 'GPP_NT_CUT_84', 'GPP_NT_CUT_95', 'GPP_NT_CUT_MEAN', 'GPP_NT_CUT_REF', 'GPP_NT_CUT_SE', 'GPP_NT_CUT_USTAR50', 'GPP_NT_VUT_05', 'GPP_NT_VUT_16', 'GPP_NT_VUT_25', 'GPP_NT_VUT_50', 'GPP_NT_VUT_75', 'GPP_NT_VUT_84', 'GPP_NT_VUT_95', 'GPP_NT_VUT_MEAN', 'GPP_NT_VUT_REF', 'GPP_NT_VUT_SE', 'GPP_NT_VUT_USTAR50', 'G_F_MDS_QC', 'H_F_MDS', 'LE_F_MDS', 'LW_IN_ERA', 'LW_IN_F', 'LW_IN_F_QC', 'LW_IN_JSB_ERA', 'LW_IN_JSB_F', 'LW_IN_JSB_F_QC', 'NEE_CUT_05', 'NEE_CUT_05_DAY', 'NEE_CUT_05_DAY_QC', 'NEE_CUT_05_NIG

Merge insitu and sentinel data


In [9]:
# FUNCTIONS
def matching (sentinel, insitu, df):
    insitu = insitu.set_index('date')
    sentinel = sentinel.set_index('date')
    # print(insitu.columns)
    # print(sentinel.columns)
    merged_input = pd.merge(left= insitu, right = sentinel, how="inner", left_index = True , right_index = True)
    # print(merged_input.index[:5])
    merged_input['date'] = pd.to_datetime(merged_input.index)
    # print(merged_input.columns)

    cols = ['date'] + [col for col in merged_input.columns if col != 'date']
    merged_input = merged_input[cols]
    start, end = merged_input['date'].iloc[[0, -1]]    
    merged_input['latitude'] = df.loc[df['station id'] == id, 'latitude'].iloc[0]
    merged_input['longitude'] = df.loc[df['station id'] == id, 'longitude'].iloc[0]
    merged_input['elevation'] = df.loc[df['station id'] == id, 'elevation'].iloc[0]
    merged_input['canopy_height'] = df.loc[df['station id'] == id, 'canopy_height'].iloc[0]
    merged_input['instrument_height'] = df.loc[df['station id'] == id, 'instrument_height'].iloc[0]
    merged_input['month'] = pd.DatetimeIndex(merged_input['date']).month
    merged_input['day'] = pd.DatetimeIndex(merged_input['date']).day
    # merged_input['ecosystem'] = df.loc[df['station id'] == id, 'main ecosystem'].iloc[0]
    # add season columns
    # Mapping months to seasons

    return merged_input, start, end
def additional_gobal(df,data):
    
    month_to_season = {
        1: 'winter', 2: 'winter', 3: 'spring',
        4: 'spring', 5: 'spring', 6: 'summer',
        7: 'summer', 8: 'summer', 9: 'fall',
        10: 'fall', 11: 'fall', 12: 'winter'
    }

    # Apply mapping
    data['season'] = data['month'].map(month_to_season)
    df_encoded = pd.get_dummies(data, columns=['season'], prefix='')
    print(df_encoded.columns)
    data[['winter', 'spring', 'summer', 'fall']] = df_encoded[['_winter', '_spring', '_summer', '_fall']].astype(int)
    df_encoded1 = pd.get_dummies(df, columns=['main ecosystem'], prefix='biom')
    df[['biom_croplands', 'biom_dbf','biom_enf', 'biom_grasslands']]=df_encoded1[['biom_croplands', 'biom_dbf','biom_enf', 'biom_grasslands']].astype(int)
    data['biom_croplands'] = df.loc[df['station id'] == id, 'biom_croplands'].iloc[0]
    data['biom_dbf'] = df.loc[df['station id'] == id, 'biom_dbf'].iloc[0]
    data['biom_enf'] = df.loc[df['station id'] == id, 'biom_enf'].iloc[0]
    data['biom_grasslands'] = df.loc[df['station id'] == id, 'biom_grasslands'].iloc[0]
    data.drop(columns = ['season'], inplace = True)
    return data

def plot_raw_vs_gap_filled(df, df_smoothed, df_raw, col, site, directory_plot, details):
    if 'date' in df.columns and 'date' in df_raw.columns:
        # Ensure the 'date' column is in datetime format
        df['date'] = pd.to_datetime(df['date'])
        df_raw['date'] = pd.to_datetime(df_raw['date'])
        
        # Set the date column as the index 
        df.set_index('date', inplace=True)
        df_raw.set_index('date', inplace=True)
    
    # Plotting
    plt.figure(figsize=(15, 5))  # Adjust the figure size as necessary
    
    # Plot gap-filled data as a line
    plt.plot(df.index, df[col], label=f'{col} Gap Filled', linestyle='--', color = 'orange')
    plt.plot(df_smoothed.index, df_smoothed[col], label=f'{col} Smoothed', linestyle='-', color = 'red')
    
    # Plot raw data as scatter
    plt.scatter(df_raw.index, df_raw[col], label=f'{col} Raw Data', marker='*', s=20, color = 'blue')  # s is the marker size

    # Adding titles and labels
    plt.title(f'Time Series of {site}')
    plt.xlabel('Date')
    plt.ylabel(col)
    plt.legend()

    # Improve layout to accommodate the x-axis date labels
    plt.gcf().autofmt_xdate()  # Auto formats the date labels to prevent overlap
    # Date Formatting and Ticks
    plt.gca().xaxis.set_major_locator(MonthLocator(bymonth=None, interval=3))  # Set ticks every 4 months
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m'))  # Set date format to Year-Month
    plt.savefig(os.path.join(directory_plot, f'{site}_{col}_s2_raw_vs_gap_filled_{details}.png'))

    plt.close()

# DECOMPOSE
col_ori =[ 'date',
 'day',
 'month',
 'latitude',
 'longitude',
 'elevation',
 'canopy_height',
 'instrument_height',
 'biom_croplands', 'biom_dbf','biom_enf', 'biom_grasslands',
 'winter', 'spring', 'summer', 'fall',
 'NEE_VUT_USTAR50_DAY', 
 'RECO_DT_VUT_USTAR50', 
 'GPP_DT_VUT_USTAR50']

col_to_decompose = ['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'CLr',
 'EVI',
 'EVI2',
 'LSWI', 
 'MNDVI',
 'MNDWI',
 'NDII',
 'NDVI',
 'H_F_MDS',
 'LE_F_MDS',
 'LW_IN_F',
 'PA_F',
 'P_F',
 'SW_IN_F',
 'TA_F',
 'VPD_F',
 'WS_F' ]

def decompose_df(df,site,directory_plot = directory_plot, col_ori = col_ori, col_to_decompose = col_to_decompose):
    # Load the data

    # Remove unwanted column if it exists
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'], inplace=True)
    if 'date.1' in df.columns:
        df.drop(columns=['date.1'], inplace=True)

    # Convert 'date' to datetime and set it as the index
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df = df.asfreq('D')  # Ensure frequency is daily

    # Prepare DataFrames for each component
    # df_trend = pd.DataFrame(index=df.index)
    # df_season = pd.DataFrame(index=df.index)
    # df_residual = pd.DataFrame(index=df.index)
    df_all = pd.DataFrame(index=df.index)
    df_all[col_ori[1:]] = df[col_ori[1:]]
    df_all[col_to_decompose] = df[col_to_decompose] 
    # df_trend[col_ori[1:]] = df[col_ori[1:]]
    # df_season[col_ori[1:]] = df[col_ori[1:]]
    # df_residual[col_ori[1:]] = df[col_ori[1:]]


    # Apply STL decomposition to each column and store the results
    stl_results = {}
    for col in col_to_decompose:
        # Set 'period' to 90 for approximately three months of seasonality
        stl = STL(df[col], seasonal=13, period=90)  
        result = stl.fit()
        stl_results[col] = result
        
        # Store each component in its respective DataFrame
        # df_trend[col] = result.trend
        # df_season[col] = result.seasonal
        # df_residual[col] = result.resid
        df_all[col+"_trend"] = result.trend
        df_all[col+"_season"] = result.seasonal
        df_all[col+"_residual"] = result.resid  
    print(f"STL Decomposition done.")

    # Optionally, plot results for each component
    for col, result in stl_results.items():
        # print(f"Plotting results for {col} of {site}")
        # plt.title(f"STL Decomposition for {col} of {site}")
        # plt.figure(figsize=(15, 6))
        # result.plot()
        plt.figure(figsize=(15, 6))  # Initialize the figure
        # plt.title(f"STL Decomposition for {col} of {site}")  # Optional: Add a title
        result.plot()
        full_path = os.path.join(directory_plot,"decomposed", f"{site}_{col}.png")  # Full path for the file
        plt.savefig(full_path)  # Save the figure
        plt.close()
    print(f"STL Decomposition plots saved")

    # return df_trend, df_season, df_residual, df_all
    return df_all


In [ ]:
details = 'fetch_desired'
# chosen by the intersections of columns from 4 sites
insitu_cols = [ 'date', 'H_F_MDS', 'LE_F_MDS', 'LW_IN_F', 'PA_F', 'P_F', 'SW_IN_F', 'TA_F', 'VPD_F', 'WS_F', 'NEE_VUT_USTAR50_DAY', 'RECO_DT_VUT_USTAR50', 'GPP_DT_VUT_USTAR50']

for id in df['station id']:
    site = df.loc[df['station id'] == id, 'site name'].iloc[0]
    print(site)
    sentinel = pd.read_csv(os.path.join(directory_sentinel,f"{site}_Vegetation_indices_processed.csv"))
    insitu = pd.read_csv(os.path.join(directory_insitu,f"ICOSETC_{id}_FLUXNET_DD_L2.csv"))
    insitu['TIMESTAMP'] = pd.to_datetime(insitu['TIMESTAMP'].astype(str), format='%Y%m%d')
    insitu.rename(columns={'TIMESTAMP':'date'}, inplace=True)
    sentinel ['date'] = pd.to_datetime(sentinel['date'])
    merged_input, start, end = matching (sentinel, insitu[insitu_cols], df)
    df1 = df.copy()
    merged_input = additional_gobal(df1,merged_input)
    # break
    # print(merged_input.columns) 
    # df_trend, df_season, df_residual, df_all = decompose_df(df,site)
    df_all = decompose_df(merged_input,site)
    df_all.to_csv(os.path.join(directory_input,"Preprocessed",f"{site}_preprocessed_{start.strftime('%d%m%Y')}_{end.strftime('%d%m%Y')}.csv"))


Estrees-Mons A28
Index(['date', 'H_F_MDS', 'LE_F_MDS', 'LW_IN_F', 'PA_F', 'P_F', 'SW_IN_F',
       'TA_F', 'VPD_F', 'WS_F', 'NEE_VUT_USTAR50_DAY', 'RECO_DT_VUT_USTAR50',
       'GPP_DT_VUT_USTAR50', 'NDVI', 'EVI', 'EVI2', 'CLr', 'MNDVI', 'MNDWI',
       'LSWI', 'NDII', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A',
       'B9', 'B11', 'B12', 'latitude', 'longitude', 'elevation',
       'canopy_height', 'instrument_height', 'month', 'day', '_fall',
       '_spring', '_summer', '_winter'],
      dtype='object')


C:\Users\karisma\AppData\Local\Temp\ipykernel_1916\1695346669.py:170: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all[col+"_residual"] = result.resid
C:\Users\karisma\AppData\Local\Temp\ipykernel_1916\1695346669.py:168: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all[col+"_trend"] = result.trend
C:\Users\karisma\AppData\Local\Temp\ipykernel_1916\1695346669.py:169: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider j

STL Decomposition done.


c:\Users\karisma\AppData\Local\anaconda3\envs\environment_GPP_new_March2023\Lib\site-packages\statsmodels\tsa\seasonal.py:345: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(len(series), 1, sharex=True)
C:\Users\karisma\AppData\Local\Temp\ipykernel_1916\1695346669.py:179: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(15, 6))  # Initialize the figure


STL Decomposition plots saved
Hohes Holz
Index(['date', 'H_F_MDS', 'LE_F_MDS', 'LW_IN_F', 'PA_F', 'P_F', 'SW_IN_F',
       'TA_F', 'VPD_F', 'WS_F', 'NEE_VUT_USTAR50_DAY', 'RECO_DT_VUT_USTAR50',
       'GPP_DT_VUT_USTAR50', 'NDVI', 'EVI', 'EVI2', 'CLr', 'MNDVI', 'MNDWI',
       'LSWI', 'NDII', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A',
       'B9', 'B11', 'B12', 'latitude', 'longitude', 'elevation',
       'canopy_height', 'instrument_height', 'month', 'day', '_fall',
       '_spring', '_summer', '_winter'],
      dtype='object')


...

<Figure size 1500x600 with 0 Axes>

<Figure size 1500x600 with 0 Axes>

<Figure size 1500x600 with 0 Axes>

<Figure size 1500x600 with 0 Axes>

<Figure size 1500x600 with 0 Axes>